# ADSP 32021
### Autumn 2023
### Group 6 Assignment 1

#### Danil Meresenchi, Ian Vogt, Monica Zhang, Jason Rajan

In [643]:
#from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import statsmodels.stats.stattools as sms
from statsmodels.compat import lzip
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score

#set pandas options
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)
#pd.set_option('precision', 2)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline
warnings.filterwarnings("ignore")

In [644]:
data= pd.read_csv("athletes.csv")

In [645]:
data.head()

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220.0,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,645.0,300.0,NaN,196.0,NaN,NaN,245.0,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,465.0,NaN,1053.0,NaN,NaN,1081.0,205.0,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|


In [646]:
data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423006 entries, 0 to 423005
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   athlete_id  423003 non-null  float64
 1   name        331110 non-null  object 
 2   region      251262 non-null  object 
 3   team        155160 non-null  object 
 4   affiliate   241916 non-null  object 
 5   gender      331110 non-null  object 
 6   age         331110 non-null  float64
 7   height      159869 non-null  float64
 8   weight      229890 non-null  float64
 9   fran        55426 non-null   float64
 10  helen       30279 non-null   float64
 11  grace       40745 non-null   float64
 12  filthy50    19359 non-null   float64
 13  fgonebad    29738 non-null   float64
 14  run400      22246 non-null   float64
 15  run5k       36097 non-null   float64
 16  candj       104435 non-null  float64
 17  snatch      97280 non-null   float64
 18  deadlift    115323 non-null  float64
 19  ba

In [647]:
#get total number of records
data.shape

(423006, 27)

In [648]:
# Remove not relevant columns
data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                           'train','background','experience','schedule','howlong', \
                           'deadlift','candj','snatch','backsq','experience',\
                           'background','schedule','howlong'])
data = data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                          'filthy50','fgonebad','run400','run5k','pullups','train'])

In [649]:
data.head()

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
6,South Central,Male,21.0,72.0,175.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amou...,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|
13,Central East,Male,43.0,71.0,185.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amount|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,2-4 years|
21,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|
22,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|
27,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|


In [650]:
data.shape

(32172, 14)

In [651]:
data = data[data['weight'] < 1500]
data = data[data['gender'] != '--']
data = data[data['age'] >= 18]
data = data[(data['height'] < 96) & (data['height'] > 48)]

data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
             & (data['deadlift'] <= 636))]

data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

In [652]:
#Dummy encode eat column
unique_outcomes = set()
for response in data['eat']:
    if '|' in response:
        outcomes = [outcome.strip() for outcome in response.split('|')]
        unique_outcomes.update(outcomes)
    else:
        unique_outcomes.add(response)

#Create columns for each unique outcome and dummy encode
for outcome in unique_outcomes:
    column_name = f"eat_{outcome}"
    data[column_name] = data['eat'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

#Drop the original 'eat' column if needed
data = data.drop(columns=['eat','eat_'])


In [653]:
#binary encode background column
unique_outcomes = set()
for response in data['background']:
    if '|' in response:
        outcomes = [outcome.strip() for outcome in response.split('|')]
        unique_outcomes.update(outcomes)
    else:
        unique_outcomes.add(response)

#Create columns for each unique outcome and dummy encode
for outcome in unique_outcomes:
    column_name = f"background_{outcome}"
    data[column_name] = data['background'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

#Drop the original 'background' column if needed
data = data.drop(columns=['background','background_'])

In [654]:
#binary encode experience column
unique_outcomes = set()
for response in data['experience']:
    if '|' in response:
        outcomes = [outcome.strip() for outcome in response.split('|')]
        unique_outcomes.update(outcomes)
    else:
        unique_outcomes.add(response)

#Create columns for each unique outcome and dummy encode
for outcome in unique_outcomes:
    column_name = f"experience_{outcome}"
    data[column_name] = data['experience'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

#Drop the original 'background' column if needed
data = data.drop(columns=['experience','experience_'])

In [655]:
#binary encode schedule column
unique_outcomes = set()
for response in data['schedule']:
    if '|' in response:
        outcomes = [outcome.strip() for outcome in response.split('|')]
        unique_outcomes.update(outcomes)
    else:
        unique_outcomes.add(response)

#Create columns for each unique outcome and dummy encode
for outcome in unique_outcomes:
    column_name = f"schedule_{outcome}"
    data[column_name] = data['schedule'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

#Drop the original 'background' column if needed
data = data.drop(columns=['schedule','schedule_'])

In [656]:
#binary encode howlong column
unique_outcomes = set()
for response in data['howlong']:
    if '|' in response:
        outcomes = [outcome.strip() for outcome in response.split('|')]
        unique_outcomes.update(outcomes)
    else:
        unique_outcomes.add(response)

#Create columns for each unique outcome and dummy encode
for outcome in unique_outcomes:
    column_name = f"howlong_{outcome}"
    data[column_name] = data['howlong'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

#Drop the original 'background' column if needed
data = data.drop(columns=['howlong','howlong_'])

In [657]:
# Creating a region dictionary for maping

region_mapping = {
    # Region 1 Group
    'Africa': 'Region_1',
    'Asia': 'Region_1',
    'Australia': 'Region_1',
    'Latin America': 'Region_1',
    
    # Region 2 Group
    'Canada East': 'Region_2',
    'Canada West': 'Region_2',

    # Region 3 Group
    'Northern California': 'Region_3',
    'Southern California': 'Region_3',
    
    # Region 4 Group
    'Central East': 'Region_4',

    # Region 5 Group
    'Mid Atlantic': 'Region_5',

    # Region 6 Group
    'North Central': 'Region_6',
    'South Central': 'Region_6',

    # Region 7 Group
    'North East': 'Region_7',
    'North West': 'Region_7',

    # Region 8 Group  
    'South East': 'Region_8',
    'South West': 'Region_8',
    
    # Region 9 Group
    'Europe': 'Region_9'
}

# Apply the mapping
data['region'] = data['region'].map(region_mapping)

In [658]:
#encode region and gender
le=LabelEncoder()
data["le_region"] = le.fit_transform(data['region'])
data["le_gender"] = le.fit_transform(data['gender'])

In [659]:
#Lets get the crosswalk for region and gender
data.groupby(['region', 'le_region']).size().reset_index(name='counts')

,region,le_region,counts
0,Region_1,0,3602
1,Region_2,1,1935
2,Region_3,2,3140
3,Region_4,3,2319
4,Region_5,4,2616
5,Region_6,5,5350
6,Region_7,6,4483
7,Region_8,7,4842
8,Region_9,8,2560


In [660]:
data.groupby(['gender', 'le_gender']).size().reset_index(name='counts')

,gender,le_gender,counts
0,Female,0,8446
1,Male,1,22401


In [661]:
#drop unencoded region/gender columns
data = data.drop(columns=['region','gender'])
#calculate our total_lift target value using clean and jerk, snatch,deadlift and backsquat values
data['total_lift'] = data['candj'] + data['snatch'] + data['deadlift'] + data['backsq']

In [662]:
#run eda on cleaned dataset
#profile = ProfileReport(data)
#profile.to_file("athletics_cleaned_profile_output.html")

In [663]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30847 entries, 21 to 422961
Data columns (total 43 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   age                                                                  30847 non-null  float64
 1   height                                                               30847 non-null  float64
 2   weight                                                               30847 non-null  float64
 3   candj                                                                30847 non-null  float64
 4   snatch                                                               30847 non-null  float64
 5   deadlift                                                             30847 non-null  float64
 6   backsq                                                               30847 non-null  float64
 7   ea

In [664]:
# Create a copy of this dataset and call it dataset_v1
dataset_v1 = data.copy()

# Write a new csv for the "uncleaned" version of the dataset
dataset_v1.to_csv('athletes_v1.csv')

In [665]:
# Apply some transformations and feature engineering
# Adding a new domain-specific feature BMI, must round then convert inches to meters and pounds to killograms.
data['BMI'] = round((data['weight'] * 0.453592) / (data['height']*0.0254)**2,1)

# Binning age, height, and weight, region, BMI index
age_bins = [0, 20, 30, 40, 50, 60, 100]
age_labels = ['0-20', '20-30', '30-40', '40-50', '50-60', '60+']  # Adjust labels as needed

height_bins = [0, 60, 65, 70, 75, 80, 100]
height_labels = ['0-60', '60-65', '65-70', '70-75', '75-80', '80+']

weight_bins = [0, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 750]
weight_labels = ['0-100', '100-125', '125-150', '150-175', '175-200', '200-225', '225-250', '250-275', '275-300',
                '300-325', '325-350', '350+']

region_bins = [0,2,5,10]
region_labels = ['0-2','2-5','6+']

BMI_bins = [0,16,20,25,30,35,39,70]
BMI_labels = ['0-16','16-20','20-25','25-30','30-35','35-39','39+']

data['binned_age'] = pd.cut(data['age'], age_bins, labels=age_labels)
data['binned_height'] = pd.cut(data['height'], height_bins, labels=height_labels)
data['binned_weight'] = pd.cut(data['weight'], weight_bins, labels=weight_labels)
data['binned_region'] = pd.cut(data['le_region'], region_bins, labels=region_labels)
data['binned_BMI'] = pd.cut(data['BMI'], BMI_bins, labels=BMI_labels)

#encode new binned variables
le=LabelEncoder()
data["le_binned_age"] = le.fit_transform(data['binned_age'])
data["le_binned_height"] = le.fit_transform(data['binned_height'])
data["le_binned_weight"] = le.fit_transform(data['binned_weight'])
data["le_binned_region"] = le.fit_transform(data['binned_region'])
data["le_binned_BMI"] = le.fit_transform(data['binned_BMI'])

# Multiply height and weight together to capture some interaction effect
data["height_times_weight"] = data["height"] * data["weight"]

# Square age
data["age_squared"] = data["age"] * data["age"]

# Square weight
data["weight_squared"] = data["weight"] * data["weight"]

# Square BMI
data['BMI_squared'] = data['BMI'] * data['BMI'] 

In [666]:
### !!! Impossible record
data[data['BMI'] == 1]



,age,height,weight,candj,snatch,deadlift,backsq,eat_I weigh and measure my food,eat_I eat quality foods but don't measure the amount,eat_I eat whatever is convenient,eat_I eat 1-3 full cheat meals per week,eat_Decline to answer,eat_I eat strict Paleo,background_I played college sports,background_I played professional sports,background_I played youth or high school level sports,background_Decline to answer,background_I regularly play recreational sports,background_I have no athletic background besides CrossFit,experience_I have had a life changing experience due to CrossFit,experience_I have completed the CrossFit Level 1 certificate course,experience_I began CrossFit with a coach (e.g. at an affiliate),experience_Decline to answer,experience_I began CrossFit by trying it alone (without a coach),experience_I have attended one or more specialty courses,experience_I train other people,schedule_I usually only do 1 workout a day,schedule_I do multiple workouts in a day 2x a week,schedule_I strictly schedule my rest days,schedule_I typically rest fewer than 4 days per month,schedule_I do multiple workouts in a day 1x a week,schedule_I typically rest 4 or more days per month,schedule_I do multiple workouts in a day 3+ times a week,schedule_Decline to answer,howlong_6-12 months,howlong_2-4 years,howlong_4+ years,howlong_Decline to answer,howlong_1-2 years,howlong_Less than 6 months,le_region,le_gender,total_lift,BMI,binned_age,binned_height,binned_weight,binned_region,binned_BMI,le_binned_age,le_binned_height,le_binned_weight,le_binned_region,le_binned_BMI,height_times_weight,age_squared,weight_squared,BMI_squared
251344,47.0,60.0,5.0,95.0,85.0,215.0,225.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,2,0,620.0,1.0,40-50,0-60,0-100,0-2,0-16,3,0,0,0,0,300.0,2209.0,25.0,1.0


In [667]:
data

,age,height,weight,candj,snatch,deadlift,backsq,eat_I weigh and measure my food,eat_I eat quality foods but don't measure the amount,eat_I eat whatever is convenient,eat_I eat 1-3 full cheat meals per week,eat_Decline to answer,eat_I eat strict Paleo,background_I played college sports,background_I played professional sports,background_I played youth or high school level sports,background_Decline to answer,background_I regularly play recreational sports,background_I have no athletic background besides CrossFit,experience_I have had a life changing experience due to CrossFit,experience_I have completed the CrossFit Level 1 certificate course,experience_I began CrossFit with a coach (e.g. at an affiliate),experience_Decline to answer,experience_I began CrossFit by trying it alone (without a coach),experience_I have attended one or more specialty courses,experience_I train other people,schedule_I usually only do 1 workout a day,schedule_I do multiple workouts in a day 2x a week,schedule_I strictly schedule my rest days,schedule_I typically rest fewer than 4 days per month,schedule_I do multiple workouts in a day 1x a week,schedule_I typically rest 4 or more days per month,schedule_I do multiple workouts in a day 3+ times a week,schedule_Decline to answer,howlong_6-12 months,howlong_2-4 years,howlong_4+ years,howlong_Decline to answer,howlong_1-2 years,howlong_Less than 6 months,le_region,le_gender,total_lift,BMI,binned_age,binned_height,binned_weight,binned_region,binned_BMI,le_binned_age,le_binned_height,le_binned_weight,le_binned_region,le_binned_BMI,height_times_weight,age_squared,weight_squared,BMI_squared
21,30.0,71.0,200.0,235.0,175.0,385.0,315.0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,2,1,1110.0,27.9,20-30,70-75,175-200,0-2,25-30,1,3,4,0,3,14200.0,900.0,40000.0,778.41
22,28.0,70.0,176.0,187.0,134.0,335.0,254.0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,910.0,25.3,20-30,65-70,175-200,NaN,25-30,1,2,4,3,3,12320.0,784.0,30976.0,640.09
27,35.0,68.0,225.0,285.0,205.0,440.0,405.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,6,1,1335.0,34.2,30-40,65-70,200-225,6+,30-35,2,2,5,2,4,15300.0,1225.0,50625.0,1169.64
50,36.0,71.0,199.0,267.0,212.0,485.0,390.0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,5,1,1354.0,27.8,30-40,70-75,175-200,2-5,25-30,2,3,4,1,3,14129.0,1296.0,39601.0,772.84
60,36.0,64.0,155.0,245.0,180.0,415.0,385.0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,1,1,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,6,1,1225.0,26.6,30-40,60-65,150-175,6+,25-30,2,1,3,2,3,9920.0,1296.0,24025.0,707.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422877,25.0,70.0,163.0,175.0,135.0,240.0,220.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,7,1,770.0,23.4,20-30,65-70,150-175,6+,20-25,1,2,3,2,2,11410.0,625.0,26569.0,547.56
422881,24.0,70.0,174.0,143.0,121.0,351.0,287.0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,902.0,25.0,20-30,65-70,150-175,NaN,20-25,1,2,3,3,2,12180.0,576.0,30276.0,625.00
422915,25.0,64.0,126.0,110.0,88.0,243.0,176.0,0,1,0,0,0,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,617.0,21.6,20-30,60-65,125-150,NaN,20-25,1,1,2,3,2,8064.0,625.0,15876.0,466.56
422923,22.0,72.0,174.0,115.0,95.0,175.0,115.0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,5,0,500.0,23.6,20-30,70-75,150-175,2-5,20-25,1,3,3,1,2,12528.0,484.0,30276.0,556.96


In [668]:
### Display Error to NaN at binned_region 3 to Adress
# nan_rows = data[data['binned_region'].isna()]
# display(nan_rows)

In [669]:
# Create a copy of this dataset and call it dataset_v2
dataset_v2 = data.copy()

# Write a new csv for the "uncleaned" version of the dataset
dataset_v2.to_csv('athletes_v2.csv')

In [670]:
#assign variables for our model
y_v1 = dataset_v1['total_lift']
x_v1 = dataset_v1[['age',
'height',
'weight',
'eat_I eat 1-3 full cheat meals per week',
"eat_I eat quality foods but don't measure the amount",
'eat_I eat strict Paleo',
'eat_Decline to answer',
'eat_I eat whatever is convenient',
'eat_I weigh and measure my food',
'background_I played college sports',
'background_I played youth or high school level sports',
'background_I have no athletic background besides CrossFit',
'background_Decline to answer',
'background_I regularly play recreational sports',
'background_I played professional sports',
'experience_I train other people',
'experience_I have had a life changing experience due to CrossFit',
'experience_I began CrossFit by trying it alone (without a coach)',
'experience_I have completed the CrossFit Level 1 certificate course',
'experience_Decline to answer',
'experience_I began CrossFit with a coach (e.g. at an affiliate)',
'experience_I have attended one or more specialty courses',
'schedule_I do multiple workouts in a day 1x a week',
'schedule_I do multiple workouts in a day 3+ times a week',
'schedule_I do multiple workouts in a day 2x a week',
'schedule_I typically rest fewer than 4 days per month',
'schedule_I strictly schedule my rest days',
'schedule_I typically rest 4 or more days per month',
'schedule_Decline to answer',
'schedule_I usually only do 1 workout a day',
'howlong_4+ years',
'howlong_Less than 6 months',
'howlong_6-12 months',
'howlong_2-4 years',
'howlong_Decline to answer',
'howlong_1-2 years',
'le_region',
'le_gender'
]]

In [671]:
#assign variables for our model
y_v2 = dataset_v2['total_lift']
x_v2 = dataset_v2[['age_squared',
'height',
'weight',
'eat_I eat 1-3 full cheat meals per week',
"eat_I eat quality foods but don't measure the amount",
'eat_I eat strict Paleo',
'eat_Decline to answer',
'eat_I eat whatever is convenient',
'eat_I weigh and measure my food',
'background_I played college sports',
'background_I played youth or high school level sports',
'background_I have no athletic background besides CrossFit',
'background_Decline to answer',
'background_I regularly play recreational sports',
'background_I played professional sports',
'experience_I train other people',
'experience_I have had a life changing experience due to CrossFit',
'experience_I began CrossFit by trying it alone (without a coach)',
'experience_I have completed the CrossFit Level 1 certificate course',
'experience_Decline to answer',
'experience_I began CrossFit with a coach (e.g. at an affiliate)',
'experience_I have attended one or more specialty courses',
'schedule_I do multiple workouts in a day 1x a week',
'schedule_I do multiple workouts in a day 3+ times a week',
'schedule_I do multiple workouts in a day 2x a week',
'schedule_I typically rest fewer than 4 days per month',
'schedule_I strictly schedule my rest days',
'schedule_I typically rest 4 or more days per month',
'schedule_Decline to answer',
'schedule_I usually only do 1 workout a day',
'howlong_4+ years',
'howlong_Less than 6 months',
'howlong_6-12 months',
'howlong_2-4 years',
'howlong_Decline to answer',
'howlong_1-2 years',
'le_region',
'le_gender'
]]

In [672]:
# Split the X and y from dataset v2 into train and test using the same test_size and random_state as before
X_train_v1, X_test_v1, y_train_v1, y_test_v1 = train_test_split(x_v1, y_v1, test_size=0.2, random_state=42)
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(x_v2, y_v2, test_size=0.2, random_state=42)

In [673]:
X_train_v1 = sm.add_constant(X_train_v1)

model_v1 = sm.OLS(y_train_v1,X_train_v1)
results_v1 = model_v1.fit(cov_type="HC0")
#results1 = model1.fit()
#results1 = model1.fit(cov_type='cluster', cov_kwds={'groups': df["tri_nw_dmy"]})

#results = model.fit() #The model then needs to be fitted
print(results_v1.summary())

                            OLS Regression Results                            
Dep. Variable:             total_lift   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     1924.
Date:                Wed, 18 Oct 2023   Prob (F-statistic):               0.00
Time:                        05:57:01   Log-Likelihood:            -1.5858e+05
No. Observations:               24677   AIC:                         3.172e+05
Df Residuals:                   24638   BIC:                         3.175e+05
Df Model:                          38                                         
Covariance Type:                  HC0                                         
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------

In [674]:
#get RMSE metric for testing sample
X_test_v1 = sm.add_constant(X_test_v1)
ypred_v1 = results_v1.predict(X_test_v1)
rmse_v1 = rmse(y_test_v1, ypred_v1)
print(rmse_v1)
r2_v1 = r2_score(y_test_v1, ypred_v1)
print(r2_v1)

151.1195996312935
0.706529134430423


In [675]:
X_train_v2 = sm.add_constant(X_train_v2)

model_v2 = sm.OLS(y_train_v2,X_train_v2)
results_v2 = model_v2.fit(cov_type="HC0")
#results1 = model1.fit()
#results1 = model1.fit(cov_type='cluster', cov_kwds={'groups': df["tri_nw_dmy"]})

#results = model.fit() #The model then needs to be fitted
print(results_v2.summary())

                            OLS Regression Results                            
Dep. Variable:             total_lift   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     1923.
Date:                Wed, 18 Oct 2023   Prob (F-statistic):               0.00
Time:                        05:57:04   Log-Likelihood:            -1.5857e+05
No. Observations:               24677   AIC:                         3.172e+05
Df Residuals:                   24638   BIC:                         3.175e+05
Df Model:                          38                                         
Covariance Type:                  HC0                                         
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------

In [676]:
#get RMSE metric for testing sample
X_test_v2 = sm.add_constant(X_test_v2)
ypred_v2 = results_v2.predict(X_test_v2)
rmse_v2 = rmse(y_test_v2, ypred_v2)
print(rmse_v2)
r2_v2 = r2_score(y_test_v2, ypred_v2)
print(r2_v2)

151.05488793704058
0.7067804179284656
